In [ ]:
import torch

# Define time steps
T = 3  # Example 3 time periods
device = (
    torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
)  # Use "cuda" if available

# Initialize decision variables as trainable parameters
P11 = torch.rand(T, requires_grad=True, device=device)
P12 = torch.rand(T, requires_grad=True, device=device)
P21 = torch.rand(T, requires_grad=True, device=device)
P22 = torch.rand(T, requires_grad=True, device=device)
U1 = torch.rand(T, requires_grad=True, device=device)
U2 = torch.rand(T, requires_grad=True, device=device)

# Constants (example values)
P1T = torch.tensor([100, 120, 110], dtype=torch.float, device=device)
P2T = torch.tensor([90, 80, 100], dtype=torch.float, device=device)
PD1 = torch.tensor([80, 100, 90], dtype=torch.float, device=device)
PD2 = torch.tensor([70, 60, 100], dtype=torch.float, device=device)

L_d11, L_d12, L_d21, L_d22 = 0.95, 0.90, 0.92, 0.93  # Loss factors
k1 = torch.tensor([1.0, 1.1, 1.2], dtype=torch.float, device=device)
k2 = torch.tensor([1.3, 1.2, 1.0], dtype=torch.float, device=device)
E1 = torch.tensor([10, 10, 10], dtype=torch.float, device=device)
E2 = torch.tensor([10, 10, 10], dtype=torch.float, device=device)

# Hyperparameters for constraint penalties
λ_s = 100  # Supply penalty
λ_d = 100  # Demand penalty
λ_n = 100  # Non-negativity penalty

# Define optimizer
optimizer = torch.optim.Adam([P11, P12, P21, P22, U1, U2], lr=0.01)

# Gradient Descent Loop
for epoch in range(5000):
    optimizer.zero_grad()

    # Cost function
    J = torch.sum(k1 * (P11 + P12) + k2 * (P21 + P22) + E1 * U1 + E2 * U2)

    # Supply constraint penalties
    L_supply = λ_s * torch.sum(
        torch.relu(P11 + P12 - P1T) ** 2 + torch.relu(P21 + P22 - P2T) ** 2
    )

    # Demand constraint penalties
    L_demand = λ_d * torch.sum(
        (P11 * L_d11 + P21 * L_d21 + U1 - PD1) ** 2
        + (P12 * L_d12 + P22 * L_d22 + U2 - PD2) ** 2
    )

    # Non-negativity penalty
    L_neg = λ_n * torch.sum(
        torch.relu(-P11) ** 2
        + torch.relu(-P12) ** 2
        + torch.relu(-P21) ** 2
        + torch.relu(-P22) ** 2
        + torch.relu(-U1) ** 2
        + torch.relu(-U2) ** 2
    )

    # Total loss
    L_total = J + L_supply + L_demand + L_neg

    # Compute gradients and update
    L_total.backward()
    optimizer.step()

    # Print every 500 iterations
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {L_total.item()}")

# Print final results
print("\nOptimized Power Dispatch:")
print(f"P11: {P11.detach().cpu().numpy()}")
print(f"P12: {P12.detach().cpu().numpy()}")
print(f"P21: {P21.detach().cpu().numpy()}")
print(f"P22: {P22.detach().cpu().numpy()}")
print(f"U1:  {U1.detach().cpu().numpy()}")
print(f"U2:  {U2.detach().cpu().numpy()}")

Epoch 0, Loss: 4166241.75
Epoch 500, Loss: 2931738.0
Epoch 1000, Loss: 1997605.875
Epoch 1500, Loss: 1307891.75
Epoch 2000, Loss: 815246.5625
Epoch 2500, Loss: 478400.25
Epoch 3000, Loss: 260638.890625
Epoch 3500, Loss: 129530.6015625
Epoch 4000, Loss: 57509.24609375
Epoch 4500, Loss: 22500.822265625

Optimized Power Dispatch:
P11: [27.490084 32.901146 30.708857]
P12: [24.371119 21.19312  33.332634]
P21: [27.588835 33.524803 30.101   ]
P22: [25.01851  21.669188 33.138405]
U1:  [27.78087  32.805595 30.942604]
U2:  [24.56154  20.729534 33.805378]
